In [2]:
! pip install pyedgar   

  Obtaining dependency information for pyedgar from https://files.pythonhosted.org/packages/04/56/83735f5281f88f13ca13fcf30535ff5676862b3988989b7f11d57cd0b3fd/pyedgar-0.1.8-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/49.8 kB ? eta -:--:--
   ------------------------ --------------- 30.7/49.8 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------- 49.8/49.8 kB 842.4 kB/s eta 0:00:00


In [3]:
from pyedgar import Filing
from pyedgar import EDGARIndex  # import the EDGARIndex function from pyedgar pacakge for downloading EDGAR files
from bs4 import BeautifulSoup  
import requests         # use its requests.get.text function to get the text of a url link
import os
import re               # Regular Expression package for convert similar text into one 
import pandas as pd  
import time # calling the package time as it is required to limit the server request to meet EDGAR database ping limit

In [4]:
# This is the format of file type in EDGARIndex function
file_type = '8-K'

In [6]:
# This is the location of folder to save all the downloaded files
filesave_directory = r"C:\Users\sidha\OneDrive\Sid\MSF\Summer\RA\Code\8k"


Step 1: extract all the 8-K files indices from EDGAR by using EDGARIndex function imported from pyedgar package

In [9]:
# set True for first time downloading and False for future time
all_indices = EDGARIndex(force_download=False)

In [1]:
#database.head(10)  # check the first 10 observations
database.info()

NameError: name 'database' is not defined

In [7]:
print(database)  # there are totally 1,767,668 8K files 

             cik                                        name   form  \
0             20                    K TRON INTERNATIONAL INC    8-K   
1             20                    K TRON INTERNATIONAL INC    8-K   
2             20                    K TRON INTERNATIONAL INC    8-K   
3             20                    K TRON INTERNATIONAL INC    8-K   
4             20                    K TRON INTERNATIONAL INC  8-K/A   
...          ...                                         ...    ...   
1767663  1837019         Ford Credit Auto Lease Trust 2021-A    8-K   
1767664  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767665  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767666  1837985  Exeter Automobile Receivables Trust 2021-1    8-K   
1767667  1838040  Toyota Auto Receivables 2021-A Owner Trust    8-K   

          filedate             accession  
0       1995-07-10  0000893220-95-000441  
1       2001-10-17  0000893220-01-500767  
2       2002-06-26

Step 2: only keep the desired time periods and cik

In [8]:
# only get files later than 1995
# database = database.loc[(database['filedate'] > '1995-01-01')]   

In [8]:
# only get files later than 1995 and earlier than 2010
database = database.loc[(database['filedate'] > '1995-01-01') & (database['filedate'] < '2010-01-01')]   

In [10]:
# This method is wrong. 
# database = database.loc[(database['cik'] == '1750') | (database['cik'] == '1800') | (database['cik'] == '1853')]   

In [11]:
# only get files for desired CIK. We can store in a new data frame database_in. We need to replace future database by database_in. 

import numpy as np

CIK_List = [1750, 1800, 1853, 1923, 1961, 2034, 2098]
index_array =  database[database['cik'] == CIK_List[0]].index

for elem in CIK_List:
    index_array = index_array.append(database[database['cik'] == elem].index)
    
database = database.loc[index_array]

Step 3: create the new variable called 'EDGARURL', which has a format like https://www.sec.gov/Archives/edgar/data/1750/0001104659-17-062143.txt

In [12]:
database['EDGARURL'] = 'https://www.sec.gov/Archives/edgar/data/' + database['cik'].apply(str) + '/' + database['accession'].apply(str) +'.txt'

In [13]:
print(database)   # it only contains 440 observations

      cik              name form   filedate             accession  \
53   1750          AAR CORP  8-K 1997-08-04  0000895813-97-000181   
54   1750          AAR CORP  8-K 1997-12-04  0001047469-97-006771   
55   1750          AAR CORP  8-K 1997-12-11  0001047469-97-007366   
56   1750          AAR CORP  8-K 1997-12-11  0001047469-97-007486   
57   1750          AAR CORP  8-K 2002-02-20  0000895813-02-000037   
..    ...               ...  ...        ...                   ...   
900  2098  ACME UNITED CORP  8-K 2009-02-27  0001026608-09-000022   
901  2098  ACME UNITED CORP  8-K 2009-04-17  0001026608-09-000050   
902  2098  ACME UNITED CORP  8-K 2009-07-17  0001026608-09-000107   
903  2098  ACME UNITED CORP  8-K 2009-10-07  0001026608-09-000134   
904  2098  ACME UNITED CORP  8-K 2009-10-29  0001026608-09-000146   

                                              EDGARURL  
53   https://www.sec.gov/Archives/edgar/data/1750/0...  
54   https://www.sec.gov/Archives/edgar/data/1750/0...  


In [14]:
# count the total number of 8K files during the specified time period
total_entries = database['EDGARURL'].count()
print(total_entries)     # there are totally 917,359 8K files

440


Step 4: write the data index with URL links into a csv file, which will be used in Part Two.  

In [17]:
database.to_csv(r'index_filename_gen.csv', index=False)

### use the first item for experiment 

Step 5: use the get.text function from requests package to extract the text info from the 'EDGARURL' link.   

In [12]:
cik = database.iloc[0, 0]   # the CIK is in the first row and first column 
print(cik)

20


In [14]:
ccession = database.iloc[0, 4]   # the accession is in the first row and 5th column 
print(accession)

0000950159-03-000341


In [15]:
url = database.iloc[0, 5]  # the accession is in the first row and 6th column, which is the last column 
print(url)

https://www.sec.gov/Archives/edgar/data/20/0000893220-95-000441.txt


In [16]:
r = requests.get(url)
raw = r.text
print(raw)

-----BEGIN PRIVACY-ENHANCED MESSAGE-----
Proc-Type: 2001,MIC-CLEAR
Originator-Name: keymaster@town.hall.org
Originator-Key-Asymmetric:
 MFkwCgYEVQgBAQICAgADSwAwSAJBALeWW4xDV4i7+b6+UyPn5RtObb1cJ7VkACDq
 pKb9/DClgTKIm08lCfoilvi9Wl4SODbR1+1waHhiGmeZO8OdgLUCAwEAAQ==
MIC-Info: RSA-MD5,RSA,
 oQjEHoXm8Fpv1WSm3+dY8n5pfa5Sz/gHokXMu9kBC4ALszZKhsqTBfJ2DpA0VpM/
 pWmBoJSAwEKcyOL94dNy6A==

<IMS-DOCUMENT>0000893220-95-000441.txt : 19950712<IMS-HEADER>0000893220-95-000441.hdr.sgml : 19950712
ACCESSION NUMBER:		0000893220-95-000441
CONFORMED SUBMISSION TYPE:	8-K
PUBLIC DOCUMENT COUNT:		5
CONFORMED PERIOD OF REPORT:	19950623
ITEM INFORMATION:		Acquisition or disposition of assets
ITEM INFORMATION:		Other events
ITEM INFORMATION:		Financial statements and exhibits
FILED AS OF DATE:		19950710
SROS:			NONE

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			K TRON INTERNATIONAL INC
		CENTRAL INDEX KEY:			0000000020
		STANDARD INDUSTRIAL CLASSIFICATION:	INDUSTRIAL INSTRUMENTS FOR MEASUREMENT, DISPLAY, AND 

Step 6: specify the beginning point and ending point for text info extraction

In [17]:
docs_start = re.compile(r'<DOCUMENT>')
docs_end = re.compile(r'</DOCUMENT>')
docs_type = re.compile(r'<TYPE>[^\n]+')

print(docs_start)
print(docs_end)
print(docs_type)

re.compile('<DOCUMENT>')
re.compile('</DOCUMENT>')
re.compile('<TYPE>[^\\n]+')


In [18]:
doc_s = [x.end() for x in docs_start.finditer(raw)]
doc_e = [x.start() for x in docs_end.finditer(raw)]
doc_t = [x[len('<TYPE>'):] for x in docs_type.findall(raw)]

print(doc_s)
print(doc_e)
print(doc_t)

[1528, 21599, 67810, 132276, 134719]
[21577, 67788, 132254, 134697, 136172]
['8-K', 'EX-2.1', 'EX-10.1', 'EX-99.1', 'EX-99.2']


In [19]:
filetype = doc_t[0]
print(filetype)

8-K


In [22]:
document = {}
document[filetype] = raw[doc_start:doc_end]

In [23]:
print(document)

{'8-K': '2:\t\tSUITE 130\n\t\tCITY:\t\t\tCHERRY HILL\n\t\tSTATE:\t\t\tNJ\n\t\tZIP:\t\t\t08002\n</IMS-HEADER>\n<DOCUMENT>\n<TYPE>8-K\n<SEQUENCE>1\n<DESCRIPTION>K-TRON INTERNATIONAL FORM 8-K DATED 6/23/95\n<TEXT>\n\n<PAGE>   1\n\n\n\n\n\n                       SECURITIES AND EXCHANGE COMMISSION\n\n                            WASHINGTON, D.C.  20549\n\n\n\n                                    FORM 8-K\n\n\n               CURRENT REPORT PURSUANT TO SECTION 13 OR 15(d) OF\n                      THE SECURITIES EXCHANGE ACT OF 1934\n\n\n\n           Date of Report (Date of earliest event reported):      June 23, 1995 \n                                                                 ---------------\n\n                          K-TRON INTERNATIONAL, INC.\n ---------------------------------------------------------------------------\n             (Exact name of registrant specified in its charter)\n\n\n         New Jersey                   0-9576                  22-1759452    \n    -------------

In [21]:
print(document[filetype])

2:		SUITE 130
		CITY:			CHERRY HILL
		STATE:			NJ
		ZIP:			08002
</IMS-HEADER>
<DOCUMENT>
<TYPE>8-K
<SEQUENCE>1
<DESCRIPTION>K-TRON INTERNATIONAL FORM 8-K DATED 6/23/95
<TEXT>

<PAGE>   1





                       SECURITIES AND EXCHANGE COMMISSION

                            WASHINGTON, D.C.  20549



                                    FORM 8-K


               CURRENT REPORT PURSUANT TO SECTION 13 OR 15(d) OF
                      THE SECURITIES EXCHANGE ACT OF 1934



           Date of Report (Date of earliest event reported):      June 23, 1995 
                                                                 ---------------

                          K-TRON INTERNATIONAL, INC.
 ---------------------------------------------------------------------------
             (Exact name of registrant specified in its charter)


         New Jersey                   0-9576                  22-1759452    
    --------------------       --------------------       ------------------
      

Step 7: use BeautifulSoup package and get_text function to remove the html code and only keep the text

In [25]:
item_xcontent = BeautifulSoup(document[filetype], 'lxml')
print(item_xcontent)

<html><body><p>2:		SUITE 130
		CITY:			CHERRY HILL
		STATE:			NJ
		ZIP:			08002

<document>
<type>8-K
<sequence>1
<description>K-TRON INTERNATIONAL FORM 8-K DATED 6/23/95
<text>
<page>   1





                       SECURITIES AND EXCHANGE COMMISSION

                            WASHINGTON, D.C.  20549



                                    FORM 8-K


               CURRENT REPORT PURSUANT TO SECTION 13 OR 15(d) OF
                      THE SECURITIES EXCHANGE ACT OF 1934



           Date of Report (Date of earliest event reported):      June 23, 1995 
                                                                 ---------------

                          K-TRON INTERNATIONAL, INC.
 ---------------------------------------------------------------------------
             (Exact name of registrant specified in its charter)


         New Jersey                   0-9576                  22-1759452    
    --------------------       --------------------       ------------------
     

In [26]:
content = item_xcontent.get_text()
print(content)

2:		SUITE 130
		CITY:			CHERRY HILL
		STATE:			NJ
		ZIP:			08002


8-K
1
K-TRON INTERNATIONAL FORM 8-K DATED 6/23/95

   1





                       SECURITIES AND EXCHANGE COMMISSION

                            WASHINGTON, D.C.  20549



                                    FORM 8-K


               CURRENT REPORT PURSUANT TO SECTION 13 OR 15(d) OF
                      THE SECURITIES EXCHANGE ACT OF 1934



           Date of Report (Date of earliest event reported):      June 23, 1995 
                                                                 ---------------

                          K-TRON INTERNATIONAL, INC.
 ---------------------------------------------------------------------------
             (Exact name of registrant specified in its charter)


         New Jersey                   0-9576                  22-1759452    
    --------------------       --------------------       ------------------
       (State or other           (Commission File          (I.R.S. Empl

Step 8: write the text of the 8K file into a txt file and save it on the local computer. The path is directory of the folder + CIK folder + accession  

In [27]:
filename = (filesave_directory + str(cik) +'/'+ str(accession) + '.txt')
print(filename)

C:/Users/Cheng Jiang/Desktop/Research/Project with Ron/Lawyer CEO and Innovation - with Barbara/Projects/Project 20210210 8K files download from EDGAR/20/0000950159-03-000341.txt


In [28]:
os.makedirs(os.path.dirname(filename), exist_ok=True)
with open(filename, "w", encoding='utf-8') as f:
    f.write(str(content))

### Iteration through all 8K files, and only download the 8K files that have Item 5.02

In [9]:
# This code works, but the problem with this code is the error showing IndexError: list index out of range

for files in range(0, total_entries):      # iterate from the first to the last 8K file
    cik = database.iloc[files, 0]                # cik is the first variable in each file. See the printed output above 
    accession = database.iloc[files, 4]          # accession is the fifth variable in each file
    url = database.iloc[files, 5]                # url (newly created) is the sixth variable in each file

    # get the text of webpage based on url
    time.sleep(0.2) # forcing the code to wait 0.12 seconds inbetween each requests. this will ensure the code to be in limit of EDGAR serving ping limit 
    r = requests.get(url)   
    raw = r.text

    # use regular experssion to define <DOCUMENT> as the start and </DOCUMENT> as the end for text extraction
    docs_start = re.compile(r'<DOCUMENT>')
    docs_end = re.compile(r'</DOCUMENT>')
    docs_type = re.compile(r'<TYPE>[^\n]+')

    doc_s = [x.end() for x in docs_start.finditer(raw)]
    doc_e = [x.start() for x in docs_end.finditer(raw)]
    doc_t = [x[len('<TYPE>'):] for x in docs_type.findall(raw)]

    document = {}
    
    filetype = doc_t[0]    # this will assign 8-K to filetype
    
    for doc_type, doc_start, doc_end in zip(doc_t, doc_s, doc_e):
        if doc_type == filetype:
            document[filetype] = raw[doc_start:doc_end]
            break

    regex = re.compile(r'((Item|item|ITEM)(\s(5.02)))')   # use regular experssion to define the keyword for searching
    matches = regex.finditer(document[filetype])     # find the 8K files that contains Item 5.02 or item 5.02
    extrac = []
    
    for match in matches:
        extrac.append(1)

    # if it cannot find Item 5.02, then print "NO Item 5.02" on the screen and do not use BeautifulSoup to read the text.      

    if not extrac:
        print("NO Item 5.02")

        progress = 100*files/total_entries
        print(progress.round(2), "% completed")
        print("-"*100)
        continue
        
    # if it finds Item 5.02, then print "YES YES YES Item 5.02" and url on the screen. 
    # It also use BeautifulSoup to read the text and use os function to write the text into the txt file in the folder             

    else:
        print("YES YES YES Item 5.02")
        print(filetype, " : ", url)
        
        filename = (filesave_directory + str(cik) +'/'+ str(accession) + '.txt')
        item_xcontent = BeautifulSoup(document[filetype], 'lxml')
        content = item_xcontent.get_text()

        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "w", encoding='utf-8') as f:
            f.write(str(content))
        
        progress = 100*files/total_entries
        print(progress.round(2), "% completed")
        print("-"*100)

print("done")

NO Item 5.02
0.0 % completed
----------------------------------------------------------------------------------------------------
NO Item 5.02
0.0 % completed
----------------------------------------------------------------------------------------------------
NO Item 5.02
0.0 % completed
----------------------------------------------------------------------------------------------------
NO Item 5.02
0.0 % completed
----------------------------------------------------------------------------------------------------
NO Item 5.02
0.0 % completed
----------------------------------------------------------------------------------------------------


KeyboardInterrupt: 

### Modifed code

In [ ]:
for files in range(0, 10000):      # iterate from the first to the last 8K file
    cik = database.iloc[files, 0]                # cik is the first variable in each file. See the printed output above 
    accession = database.iloc[files, 4]          # accession is the fifth variable in each file
    url = database.iloc[files, 5]                # url (newly created) is the sixth variable in each file

    # get the text of webpage based on url
    r = requests.get(url)   
    raw = r.text

    # use regular experssion to define <DOCUMENT> as the start and </DOCUMENT> as the end for text extraction
    docs_start = re.compile(r'<DOCUMENT>')
    docs_end = re.compile(r'</DOCUMENT>')
    docs_type = re.compile(r'<TYPE>[^\n]+')

    doc_s = [x.end() for x in docs_start.finditer(raw)]
    doc_e = [x.start() for x in docs_end.finditer(raw)]
    doc_t = [x[len('<TYPE>'):] for x in docs_type.findall(raw)]

    document = {}
    
    filetype = doc_t[0]    # this will assign 8-K to filetype
    
    for doc_type, doc_start, doc_end in zip(doc_t, doc_s, doc_e):
        if doc_type == filetype:
            document[filetype] = raw[doc_start:doc_end]
            break

    regex = re.compile(r'((Item|item|ITEM)(\s(5.02)))')   # use regular experssion to define the keyword for searching
    matches = regex.finditer(document[filetype])     # find the 8K files that contains Item 5.02 or item 5.02
    extrac = []
    
    for match in matches:
        extrac.append(1)

    # if it cannot find Item 5.02, then print "NO Item 5.02" on the screen       

    if not extrac:
        print("NO Item 5.02")
        cont = ""
       
        progress = 100*files/total_entries
        print(progress.round(2), "% completed")
        print("-"*100)
        continue
        
    # if it finds Item 5.02, then print "YES YES YES Item 5.02" and url on the screen. 
    # It also use BeautifulSoup to read the text and use os function to write the text into the txt file in the folder             

    
    else:
        matches = regex.finditer(document[filetype])
        test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])
        test_df.columns = ['item', 'start', 'end']
        test_df['item'] = test_df.item.str.lower()
        test_df.replace('&#160;',' ',regex=True,inplace=True)
        test_df.replace('&nbsp;',' ',regex=True,inplace=True)
        test_df.replace(' ','',regex=True,inplace=True)
        test_df.replace('\.','',regex=True,inplace=True)
        test_df.replace('>','',regex=True,inplace=True)
        pos_dat = test_df.sort_values('start', ascending=True)
        pos_dat.set_index('item', inplace=True)
                
        entries = pos_dat['start'].count() - 1
        
        for entry in range(0, entries):

            item_x = document[filetype]
            item_xcontent = BeautifulSoup(item_x, 'lxml')
            content = item_xcontent.get_text()
            # content_sample = content[:60]

        filename = (filesave_directory+ str(cik) +'/'+ str(accession) + '.txt')
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "w", encoding='utf-8') as f:
            f.write(str(content))

        print("YES YES YES Item 5.02")
        print(filetype, " : ", url)

        progress = 100*files/total_entries
        print(progress.round(2), "% completed")
        print("-"*100)

print("done")



## OLD CODE

In [ ]:
for files in range(13140, total_entries):
	
	cik = database.iloc[files, 0]
	accession = database.iloc[files, 4]
	url = database.iloc[files, 5]
	
	r = requests.get(url)
	raw = r.text

	docs_start = re.compile(r'<DOCUMENT>')
	docs_end = re.compile(r'</DOCUMENT>')
	docs_type = re.compile(r'<TYPE>[^\n]+')

	doc_s = [x.end() for x in docs_start.finditer(raw)]
	doc_e = [x.start() for x in docs_end.finditer(raw)]
	doc_t = [x[len('<TYPE>'):] for x in docs_type.findall(raw)]

	document = {}
	filetype = doc_t[0]
	for doc_type, doc_start, doc_end in zip(doc_t, doc_s, doc_e):
	    if doc_type == filetype:
	        document[filetype] = raw[doc_start:doc_end]
	        break

	regex = re.compile(r'(>(Item|ITEM)(\s|&#160;|&nbsp;)(2|2A|2B|3|4|5|6)\.{0,1})|((Item|ITEM)(\s(2|3|4|5|6)))')
	print(filetype, " : ", url)

	matches = regex.finditer(document[filetype])
	extrac = []
	extract_sample = []
	for match in matches:
		extrac.append(1)

	if not extrac:
		print("NO MD&A")
		cont = ""
		filename = ('C:/EDGAR/COVID_cache/'+ str(cik) +'/'+ str(accession) + '.txt')
		os.makedirs(os.path.dirname(filename), exist_ok=True)
		with open(filename, "w", encoding='utf-8') as f:
			f.write(str(cont))

		progress = 100*files/total_entries
		print(progress.round(2), "% completed")
		print("-"*100)
		continue
	
	else:
		matches = regex.finditer(document[filetype])
		test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])
		test_df.columns = ['item', 'start', 'end']
		test_df['item'] = test_df.item.str.lower()
		test_df.replace('&#160;',' ',regex=True,inplace=True)
		test_df.replace('&nbsp;',' ',regex=True,inplace=True)
		test_df.replace(' ','',regex=True,inplace=True)
		test_df.replace('\.','',regex=True,inplace=True)
		test_df.replace('>','',regex=True,inplace=True)
		pos_dat = test_df.sort_values('start', ascending=True)
		pos_dat.set_index('item', inplace=True)

		cont_agg = []
		mda_regex = r"discussion and analysis"
		entries = pos_dat['start'].count() - 1
		for entry in range(0, entries):
			
			item_x = document[filetype][pos_dat.iloc[entry, 0]:pos_dat.iloc[entry+1, 0]]
			item_xcontent = BeautifulSoup(item_x, 'lxml')
			content = item_xcontent.get_text()
			content_sample = content[:60]

			matches = re.findall(mda_regex, content, re.IGNORECASE | re.DOTALL | re.MULTILINE)
			if not matches:
				continue
			else :
				cont_agg.append(str(content))

		cont = '[{}]'.format('\n'.join(cont_agg))
		filename = ('C:/EDGAR/COVID_10K/'+ str(cik) +'/'+ str(accession) + '.txt')
		os.makedirs(os.path.dirname(filename), exist_ok=True)
		with open(filename, "w", encoding='utf-8') as f:
			f.write(str(cont))

		progress = 100*files/total_entries
		print(progress.round(2), "% completed")
		print("-"*100)

print("done")
